In [1]:
import sys; sys.path.append('..')
import pyzx as zx
import random
zx.todd.TOPT_LOCATION = ['wsl', '/home/aleks/src/TOpt/bin/TOpt']

In [2]:
files = [
 '../circuits/Fast/Adder8_before',
 #'../circuits/Fast/adder_8_before',
 '../circuits/Fast/barenco_tof_10_before',
 '../circuits/Fast/barenco_tof_3_before',
 '../circuits/Fast/barenco_tof_4_before',
 '../circuits/Fast/barenco_tof_5_before',
 '../circuits/Fast/csla_mux_3_original_before',
 '../circuits/Fast/csum_mux_9_corrected_before',
 '../circuits/Fast/gf2^10_mult_before',
 '../circuits/Fast/gf2^4_mult_before',
 '../circuits/Fast/gf2^5_mult_before',
 '../circuits/Fast/gf2^6_mult_before',
 '../circuits/Fast/gf2^7_mult_before',
 '../circuits/Fast/gf2^8_mult_before',
 '../circuits/Fast/gf2^9_mult_before',
 '../circuits/Fast/grover_5.qc',
 '../circuits/Fast/ham15-low.qc',
 '../circuits/Fast/hwb10.qc',
 '../circuits/Fast/hwb11.qc',
 '../circuits/Fast/hwb12.qc',
 '../circuits/Fast/hwb6.qc',
 '../circuits/Fast/hwb8.qc',
 '../circuits/Fast/mod5_4_before',
 '../circuits/Fast/mod_mult_55_before',
 '../circuits/Fast/mod_red_21_before',
 '../circuits/Fast/nth_prime6.tfc',
 '../circuits/Fast/qcla_adder_10_before',
 '../circuits/Fast/qcla_com_7_before',
 '../circuits/Fast/qcla_mod_7_before',
 '../circuits/Fast/QFT16_before',
 '../circuits/Fast/QFT8_before',
 '../circuits/Fast/QFTAdd8_before',
 '../circuits/Fast/qft_4.qc',
 '../circuits/Fast/rc_adder_6_before',
 '../circuits/Fast/tof_10_before',
 '../circuits/Fast/tof_3_before',
 '../circuits/Fast/tof_4_before',
 '../circuits/Fast/tof_5_before',
 '../circuits/Fast/vbe_adder_3_before'
]

In [3]:
for f in files:
    c = zx.Circuit.load(f).to_basic_gates()
    g = c.to_graph()
    t1 = zx.tcount(g)
    zx.simplify.full_reduce(g)
    t2 = zx.tcount(g)
    zx.todd.todd_on_graph(g)
#     while True:
#         ms = zx.simplify.match_gadgets_phasepoly(g)
#         if len(ms) == 0: break
#         else: zx.simplify.apply_gadget_phasepoly(g, ms)
    print("%s: %s %s %s" % (f, t1, t2, zx.tcount(g)))

KeyboardInterrupt: 

In [ ]:
zx.tcount(g)

In [ ]:
c = zx.Circuit(3)
c.add_gate(zx.circuit.T(2, adjoint=True))
c.add_gate(zx.circuit.Tofolli(0, 1, 2))
c.add_gate(zx.circuit.T(2))
c.add_gate(zx.circuit.Tofolli(0, 1, 2))
c.add_gate(zx.circuit.T(2))
c.gates

In [ ]:
g = c.to_basic_gates().to_graph()
print(zx.tcount(g))
zx.d3.draw(g)
zx.full_reduce(g)
print(zx.tcount(g))
zx.d3.draw(g)

In [4]:
26/31.

0.8387096774193549

In [ ]:
c = zx.Circuit(2)
c.add_gate("CNOT",0,1)
c.add_gate("NOT",0)
c.add_gate("T",1)
c.add_gate("HAD",1)
c.add_gate("T",1)
c.add_gate("HAD",1)
c.add_gate("T",1,adjoint=True)
c.add_gate("CNOT",0,1)
c.add_gate("NOT",0)
c.add_gate("T",1)
c.add_gate("HAD",1)
c.add_gate("T",1,adjoint=True)
c.add_gate("HAD",1)
c.add_gate("T",1,adjoint=True)
for i in range(4): c.add_circuit(c)

In [ ]:
print(c.stats())

In [18]:
c2 = zx.Circuit.load(r'..\circuits\Fast\adder_8_before')
g = c2.to_graph()
# zx.spider_simp(g)
# zx.to_gh(g)
# zx.spider_simp(g)
zx.d3.draw(g)

In [ ]:
# should be self-adjoint
c = zx.Circuit(2)
c.add_gate("NOT",0)
c.add_gate("T",1,adjoint=True)
c.add_gate("S",1,adjoint=True)
c.add_gate("HAD",1)
c.add_gate("T",1,adjoint=True)
c.add_gate("CNOT",0,1)
c.add_gate("NOT",0)
c.add_gate("T",1)
c.add_gate("HAD",1)
c.add_gate("S",1)
c.add_gate("T",1)
c.add_gate("CNOT",0,1)

In [ ]:
g1 = c.to_graph()
zx.spider_simp(g1); zx.to_gh(g1); zx.spider_simp(g1)
#zx.full_reduce(g1)
gad = zx.gadgetize(g1)
zx.pivot_simp(g1, matchf=lambda e: not (g1.edge_s(e) in gad[0] or g1.edge_t(e) in gad[0]))
zx.spider_simp(g1)
zx.id_simp(g1)
#gad = zx.gadgetize(g1)
#zx.pivot_simp(g1, matchf=lambda e: not (g1.edge_s(e) in gad[0] or g1.edge_t(e) in gad[0]))

#zx.pivot_simp(g1)
#g1.set_qubit(11, 4)
#g1.set_qubit(17, 4)
g1.normalise()
zx.draw(g1, labels=True)

In [ ]:
g2 = g1.copy()
for e in g2.edges(): g2.set_edge_type(e, 1)
zx.quantomatic.edit_graph(g2)

In [ ]:
g3 = Out[26]

In [ ]:
zx.draw(g)

In [ ]:
def check(g1, gadgets):
    return sum(1 for v in gadgets for w in gadgets if g1.connected(v,w))//2

def phase_simp(g1):
    zx.spider_simp(g1, quiet=True)
    zx.to_gh(g1)
    #hile True:
    for i in range(10):
        zx.simplify.clifford_simp(g1, quiet=True)
        #zx.simplify.clifford_simp(g1, quiet=True)
        gadgets = set(zx.gadgetize(g1))
        zx.simplify.pivot_simp(g1, matchf=lambda e: not (g1.edge_s(e) in gadgets or g1.edge_t(e) in gadgets), quiet=True)
        #if check(g1, gadgets) == 0: break
        print(check(g1, gadgets), end='.')
    g1.normalise()
    return gadgets


In [ ]:
for i in range(20):
    g = zx.generate.cliffordT(5, 400, p_t=0.4, p_hsh=0)
    print('trying graph...')
    gadgets = phase_simp(g)
    print('done')
    #print(check(g, gadgets))

In [ ]:
g1 = g.copy()
zx.spider_simp(g1)
print(zx.t_count(g1))
zx.to_gh(g1)
for i in range(2):
    zx.simplify.clifford_simp(g1)
    zx.simplify.clifford_simp(g1)
    gadgets = set(zx.gadgetize(g1))
    zx.simplify.pivot_simp(g1, matchf=lambda e: not (g1.edge_s(e) in gadgets or g1.edge_t(e) in gadgets))
g1.normalise()
print(zx.t_count(g1))
zx.draw(g1, labels=True, figsize=(8,4))

In [ ]:
gadgets

In [ ]:
all(not g1.connected(v,w) for v in gadgets for w in gadgets)

In [ ]:
g2 = g1.copy()
for e in g2.edges(): g2.set_edge_type(e, 1)

In [ ]:
g2 = zx.quantomatic.edit_graph(g2)

In [ ]:
zx.draw(g2, labels=True)

In [ ]:
zx.extract.cut_rank(g2, [12,3,9,15, 7, 8, 16, 17], [6, 4,10,1,5])